## Sources
- Election: https://dataverse.harvard.edu/api/access/datafile/3641280?format=original&gbrecs=true
- GDP     : https://apps.bea.gov/regional/zip/CAGDP2.zip
- County  : https://www.nrcs.usda.gov/wps/portal/nrcs/detail/national/home/?cid=nrcs143_013697


In [1]:
import pandas as pd
import os

"""
.
├── etl.ipynb
├── input
│   ├── CAGDP2__ALL_AREAS_2001_2018.csv
│   ├── countypres_2000-2016.csv
│   └── fips_name_state.csv
├── load.sql
└── output
    ├── candidate.csv
    ├── county.csv
    ├── election.csv
    ├── gdp.csv
    └── industry.csv
"""

CURR_DIR = os.getcwd()
INPUT_DIR = os.path.join(CURR_DIR, "input")
OUTPUT_DIR = os.path.join(CURR_DIR, "output")

ELECTION_INFILE = os.path.join(INPUT_DIR, "countypres_2000-2016.csv")
GDP_INFILE = os.path.join(INPUT_DIR, "CAGDP2__ALL_AREAS_2001_2018.csv")
COUNTY_INFILE = os.path.join(INPUT_DIR, "fips_name_state.csv")

ELECTION_OUTFILE = os.path.join(OUTPUT_DIR, "election.csv")
COUNTY_OUTFILE = os.path.join(OUTPUT_DIR, "county.csv")
GDP_OUTFILE = os.path.join(OUTPUT_DIR, "gdp.csv")
CANDIDATE_OUTFILE = os.path.join(OUTPUT_DIR, "candidate.csv")
INDUSTRY_OUTFILE = os.path.join(OUTPUT_DIR, "industry.csv")

## Process County Data

In [2]:
county = pd.read_csv(COUNTY_INFILE, dtype=str)
county = county.fillna(value='\\N')

# dupes to delete
duplicates = [("66010","Cocos Island"), ("60020", "Ofu"), ("60020", "Olosega"), ("69120", "Aguijan")]
county = county[~pd.Series(list(zip(county['FIPS'], county['County']))).isin(duplicates)]
county.to_csv(COUNTY_OUTFILE, sep=",", header=False, index=False)

## Process Election Data

In [3]:
election = pd.read_csv(ELECTION_INFILE, dtype=str,na_values=["","NA"])
election = election.fillna(value='\\N')

# remove 48 rows for 'NA' FIPS
election = election[election.FIPS != '\\N']

# remove 656 rows (41 FIPS) of district election results (except DC)
election = election[~(election.county.str.contains("District ") & ~election.county.str.contains("Columbia"))] 

# pad the FIPS with zero if needed (format=00000)
election['FIPS'] = election['FIPS'].apply(lambda x: str(x).zfill(5) if str(x) != "\\N" else "\\N")

# remove 28 rows where the FIPS [36000, 08014] not in county FIPS
election = election[election.FIPS.isin(county.FIPS)]

# capitalize party names, drop unneeded columns
election['party'] = election['party'].apply(lambda x: x.capitalize() if x != "\\N" else "\\N")
election = election[['year', 'FIPS','candidate','party','candidatevotes']]


# create new DF to create Candidate table
candidate = election[['candidate','party']].copy()
candidate = candidate.drop_duplicates().reset_index(drop=True)
candidate.insert(0, 'CANDIDATE_ID', candidate.index)
candidate.to_csv(CANDIDATE_OUTFILE, sep=",", header=False, index=False)


# join DFs to get election with CANDIDATE_ID
election2 = pd.merge(election.copy(), candidate.copy(), on=['candidate','party'], how='left')
# reorder and drop columns
election2 = election2[['year', 'FIPS', 'CANDIDATE_ID', 'candidatevotes']]
# re-sort, reindex, then export
election2 = election2.sort_values(by=['year', 'FIPS','CANDIDATE_ID']).reset_index(drop=True)
election2.to_csv(ELECTION_OUTFILE, sep=",", header=False, index=False)

## Process GDP Data

In [4]:
# seems to fail if engine params not supplied, might be a Windows thing
gdp = pd.read_csv(GDP_INFILE, dtype=str, na_values=["","NA","(NA)", "(D)"], engine='python')  

# drop unused columns, remove footnote
gdp = gdp.drop(['GeoName','Region','TableName', 'LineCode', 'IndustryClassification', 'Unit'], axis=1)
gdp = gdp.iloc[:-4]
# remove surrounding quotes, replace nulls, trim spaces
gdp['GeoFIPS'] = gdp['GeoFIPS'].apply(lambda x: x[2:-2] if x != "\\N" else "\\N")
gdp = gdp.fillna(value='\\N')
gdp['Description'] = gdp['Description'].str.strip()

# remove 2040 rows = 60 regional FIPS(ending with 000) * 34 industry
gdp = gdp[~gdp.GeoFIPS.str.endswith("000")]
# remove 1088 rows = 32 unique FIPS not in county FIPS * 34 industry
# duplicates = list(gdp[~gdp.GeoFIPS.isin(county.FIPS)][['GeoFIPS']].drop_duplicates()['GeoFIPS'])
gdp = gdp[gdp.GeoFIPS.isin(county.FIPS)]


# create new DF to create Industry table
industry = gdp[['Description']].copy()
industry = industry.drop_duplicates().reset_index(drop=True)
industry.insert(0, 'INDUSTRY_ID', industry.index)
industry.to_csv(INDUSTRY_OUTFILE, sep=",", header=False, index=False)


# join DFs to get gdp with INDUSTRY_ID
gdp = pd.merge(gdp.copy(), industry.copy(), on=['Description'], how='left')
# reorder, drop columns
gdp = gdp[["GeoFIPS", "INDUSTRY_ID"] + [str(year) for year in range(2001, 2019)]]
# re-sort, reindex
gdp = gdp.sort_values(by=['GeoFIPS', 'INDUSTRY_ID']).reset_index(drop=True)
# transform year as a single attribute
gdp = pd.melt(gdp, id_vars=['GeoFIPS', 'INDUSTRY_ID'], var_name='YEAR', value_name='GDP')
gdp.to_csv(GDP_OUTFILE, sep=",", header=False, index=False)

## Preprocessing Summary

- County Dataset
    - Total Rows     : 3232
    - Duplicate FIPS : 4 rows
    - Final Row Count: 3228
- Election Dataset
    - Total Rows: 50524
    - N/A FIPS   : 48 rows
    - Non-county election result: 656 rows (41 FIPS)
    - FIPS not in County dataset: 28 rows (2 FIPS)
    - Final Row Count: 49792
- GDP Dataset
    - Total Rows: 107984
    - Regional FIPS: 2040 rows (60 FIPS)
    - FIPS not in County dataset: 1088 rows (32 FIPS)
    - Wide-to-tall Transform = (107984 - 2040 - 1088) * 18 years = 1887408
    - Final Row Count: 1887408

In [5]:
county

,FIPS,County,State
0,01001,Autauga,AL
1,01003,Baldwin,AL
2,01005,Barbour,AL
3,01007,Bibb,AL
4,01009,Blount,AL
...,...,...,...
3227,72151,Yabucoa,PR
3228,72153,Yauco,PR
3229,78010,St. Croix,VI
3230,78020,St. John,VI


In [6]:
election

,year,FIPS,candidate,party,candidatevotes
0,2000,01001,Al Gore,Democrat,4942
1,2000,01001,George W. Bush,Republican,11993
2,2000,01001,Ralph Nader,Green,160
3,2000,01001,Other,\N,113
4,2000,01003,Al Gore,Democrat,13997
...,...,...,...,...,...
49854,2016,56043,Donald Trump,Republican,2911
49855,2016,56043,Other,\N,371
49856,2016,56045,Hillary Clinton,Democrat,299
49857,2016,56045,Donald Trump,Republican,3033


In [7]:
gdp

,GeoFIPS,INDUSTRY_ID,YEAR,GDP
0,01001,0,2001,758696
1,01001,1,2001,661963
2,01001,2,2001,14612
3,01001,3,2001,1458
4,01001,4,2001,19518
...,...,...,...,...
1887403,56045,29,2018,19634
1887404,56045,30,2018,14912
1887405,56045,31,2018,\N
1887406,56045,32,2018,144084


In [8]:
candidate

,CANDIDATE_ID,candidate,party
0,0,Al Gore,Democrat
1,1,George W. Bush,Republican
2,2,Ralph Nader,Green
3,3,Other,\N
4,4,John Kerry,Democrat
5,5,Barack Obama,Democrat
6,6,John McCain,Republican
7,7,Mitt Romney,Republican
8,8,Hillary Clinton,Democrat
9,9,Donald Trump,Republican


In [9]:
industry

,INDUSTRY_ID,Description
0,0,All industry total
1,1,Private industries
2,2,"Agriculture, forestry, fishing and hunting"
3,3,"Mining, quarrying, and oil and gas extraction"
4,4,Utilities
5,5,Construction
6,6,Manufacturing
7,7,Durable goods manufacturing
8,8,Nondurable goods manufacturing
9,9,Wholesale trade


## Load to DB

In [10]:
q = \
f"""
/* INIT */
DROP DATABASE IF EXISTS election;
CREATE DATABASE election;
USE election;




/* DDL */
CREATE TABLE Candidate(
    CANDIDATE_ID INT PRIMARY KEY,
    CANDIDATE_NAME VARCHAR(256),
    PARTY VARCHAR(256)
);
CREATE TABLE County(
    FIPS VARCHAR(5) PRIMARY KEY,
    NAME VARCHAR(256),
    STATE VARCHAR(2)
);
CREATE TABLE Election(
    YEAR INT,
    FIPS VARCHAR(5), 
    CANDIDATE_ID INT,
    CANDIDATE_VOTES INT, 
    PRIMARY KEY(YEAR, FIPS, CANDIDATE_ID),
    FOREIGN KEY (FIPS) REFERENCES County(FIPS),
    FOREIGN KEY (CANDIDATE_ID) REFERENCES Candidate(CANDIDATE_ID)
);
CREATE TABLE Industry(
    INDUSTRY_ID INT PRIMARY KEY,
    NAME VARCHAR(256)
);
CREATE TABLE GDP(
    FIPS VARCHAR(5), 
    INDUSTRY_ID INT, 
    YEAR INT,
    GDP INT,
    PRIMARY KEY(FIPS, INDUSTRY_ID, YEAR),
    FOREIGN KEY (FIPS) REFERENCES County(FIPS),
    FOREIGN KEY (INDUSTRY_ID) REFERENCES Industry(INDUSTRY_ID)
);




/* LOAD DATA */
LOAD DATA LOCAL INFILE '{CANDIDATE_OUTFILE}' INTO TABLE Candidate 
FIELDS TERMINATED BY ',' OPTIONALLY ENCLOSED BY '"';

LOAD DATA LOCAL INFILE '{COUNTY_OUTFILE}' INTO TABLE County
FIELDS TERMINATED BY ',' OPTIONALLY ENCLOSED BY '"';

LOAD DATA LOCAL INFILE '{INDUSTRY_OUTFILE}' INTO TABLE Industry
FIELDS TERMINATED BY ',' OPTIONALLY ENCLOSED BY '"';

LOAD DATA LOCAL INFILE '{GDP_OUTFILE}' INTO TABLE GDP
FIELDS TERMINATED BY ',' OPTIONALLY ENCLOSED BY '"';

LOAD DATA LOCAL INFILE '{ELECTION_OUTFILE}' INTO TABLE Election
FIELDS TERMINATED BY ',' OPTIONALLY ENCLOSED BY '"';
"""

SCRIPT_FILE = "load.sql"
script = open(SCRIPT_FILE, "w")
script.write(q)
script.close()


#!mysql -h cis550-aws-mysql.clfnqamsaklk.us-east-1.rds.amazonaws.com -u islong --password="islong" --local-infile election < $SCRIPT_FILE;